<a href="https://colab.research.google.com/github/JuanBailke/Transfer-Learning-DIO/blob/main/Transfer_Learning_DIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning com TensorFlow/Keras no Colab - Gatos vs Cachorros

### Projeto criado para Treinamento de Redes Neurais com Transfer Learning do bootcamp [BairesDev - Machine Learning Training](https://web.dio.me/track/bairesdev-machine-learning-training)

## 1. Download e extração do dataset

In [4]:
!wget -O cats_and_dogs.zip "https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip"
!unzip -q cats_and_dogs.zip -d dataset/

--2025-08-03 15:47:26--  https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.30.24.206, 2a02:26f0:6d00:39f::317f, 2a02:26f0:6d00:3b6::317f
Connecting to download.microsoft.com (download.microsoft.com)|184.30.24.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘cats_and_dogs.zip’

cats_and_dogs.zip   100%[===================>] 786.67M   216MB/s    in 3.7s    

2025-08-03 15:47:30 (212 MB/s) - ‘cats_and_dogs.zip’ saved [824887076/824887076]



## 2. Organização das pastas (train / val split)

In [5]:
import os, shutil, random
from PIL import Image

src = 'dataset/PetImages'
dst = 'dataset_split'
os.makedirs(dst, exist_ok=True)

for label in ['Cat', 'Dog']:
    files = os.listdir(os.path.join(src, label))
    files = [f for f in files if os.path.isfile(os.path.join(src, label, f))]
    random.shuffle(files)
    split = int(0.8 * len(files))
    for phase, subset in [('train', files[:split]), ('val', files[split:])]:
        outdir = os.path.join(dst, phase, label)
        os.makedirs(outdir, exist_ok=True)
        for fname in subset:
            fpath = os.path.join(src, label, fname)
            try:
                with Image.open(fpath) as img:
                  img.verify() # verifica se o arquivo é uma imagem válida
                shutil.copy(fpath, outdir)
            except Exception as e:
                print(f"Imagem inválida removida: {fpath}")

Imagem inválida removida: dataset/PetImages/Cat/Thumbs.db
Imagem inválida removida: dataset/PetImages/Cat/666.jpg
Imagem inválida removida: dataset/PetImages/Dog/Thumbs.db


/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Imagem inválida removida: dataset/PetImages/Dog/11702.jpg


In [6]:
def remover_imagens_invalidas(pasta):
    for root, _, files in os.walk(pasta):
        for f in files:
            path = os.path.join(root, f)
            try:
                with Image.open(path) as img:
                    img.verify()
            except:
                print("Removendo imagem inválida:", path)
                os.remove(path)

remover_imagens_invalidas("dataset_split")

## 3. Importação de bibliotecas

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image

## 4. Preparação dos dados

In [8]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [9]:
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

datagen_val = ImageDataGenerator(rescale=1./255)

dirs = {
    'train': 'dataset_split/train',
    'val': 'dataset_split/val'
}

data_train = datagen_train.flow_from_directory(
    dirs['train'],
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

data_val = datagen_val.flow_from_directory(
    dirs['val'],
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 19996 images belonging to 2 classes.
Found 5002 images belonging to 2 classes.


## 5. Modelo com transfer learning (MobileNetV2)

In [10]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


## 6. Treinamento

In [ ]:
history = model.fit(
    data_train,
    validation_data=data_val,
    epochs=5
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
291/625 ━━━━━━━━━━━━━━━━━━━━ 9:42 2s/step - accuracy: 0.8980 - loss: 0.2319

## 7. Avaliação e visualização

In [ ]:
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.title('Acurácia por época')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

## 8. Predição de exemplo

In [ ]:
sample_path = data_val.filepaths[0]
img = image.load_img(sample_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img) / 255.
img_array = np.expand_dims(img_array, axis=0)

pred = model.predict(img_array)[0][0]
print(f"Imagem: {sample_path}\nPrevisão: {'Dog' if pred > 0.5 else 'Cat'} (confiança: {pred:.2f})")